# Computational Theory

## Introduction

This notebook implements key components of SHA-256 as defined in the Secure Hash Standard (FIPS 180-4).  
SHA-256 is a cryptographic hash function widely used in computing including Bitcoin, SSL/TLS Certification, digital signatures, passwords etc.

### Purpose
The implementation demonstrates understanding of:
- Bitwise logical operations and bit manipulation
- Constant generation
- Message padding for fixed block cyphers

### Structure

This notebook is organised into five problems per the assignment requirements.  
1. Binary Words & Operations - Implementing the seven logical functions used in SHA-256
2. Fractional Parts of Cube Roots - Generating 64 constants from prime number cube roots
3. Padding - Message processing with proper padding to ensure 512-bit blocks
4. Hashes - 
5. Passwords - 
- Technical Glossary - Section for defining operations & common terms used across multiple problems to prevent repeating definitions constantly.

### References
[FIPS 180-4: Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)  
[Markdown Style Guidance](https://www.markdownguide.org)  
[PEP 257: Docstring Conventions](https://peps.python.org/pep-0257/)  
[NumPy Documentation](https://numpy.org/doc/stable/)  

Additional references provided in-line throughout the notebook.



## Imports

In [1]:
import numpy as np
from typing import Generator 

## Technical Glossary

This section defines operations & technical terms used throughout the implementation of solutions for the problems set out in the assignment.  

### Bitwise Logical Operations
**Bitwise Operations** are operations that work on the individiual bits of a binary number. Applying the operator to each bit individually but all bits simulatenously.

**XOR (Exclusive OR)**  
Outputs 1 when the input bits are different.  

| A | B | A ⊕ B |
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   1   |
| 1 | 0 |   1   |
| 1 | 1 |   0   |

**AND**  
Outputs 1 when both input bits are 1  

| A | B | A & B |
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   0   |
| 1 | 0 |   0   |
| 1 | 1 |   1   |

**NOT**  
Output flips the bits  

| A | ¬A |
|---|----|
| 0 | 1  |
| 1 | 0  |

### Bit Manipulation Operations  

- `ROTR^n(x)`: Rotate right -  shifts bits to the right **n** times, 'fallen' bits move back to the left (start)
- `SHR^n(x)`: Logical shift right - shifts bits to the right **n** times; fills left with zeros  


### Common Terms

- **Bitwise Operation**: Performs an operation on each individual bit 
- **32-bit unsigned integer**: An integer represented by 32 individual bits in a binary sequence. "Unsigned" meaning non-negative values.
- **Diffusion**: Where the input changes are spread throughout the output  

See: [FIPS 180-4, Sections 2.2.2 and 3.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

## Problem 1 - Binary Words & Operations

### Implementing Parity

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\text{Parity}(x, y, z) = x \oplus y \oplus z$
>

  
**How it works**:  
Chains XOR operations across three 32-bit integers.  
Since XOR is a binary operation it takes 2 inputs. The expression is evaluated as $(x \oplus y) \oplus z$. The result of x XOR y is then, result XOR z.  

Each output bit is 1 if there is an odd number of input bits set to 1, otherwise 0. This is why its called "Parity" it checks the parity(odd/even count) of bits at each position.


**Example**  
X       =   0110  
Y       =   1110  
Z       =   0010  

Output  =   1010

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [2]:
def parity(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Parity: output bit is 1 if an odd number of input bits are 1.

    Parameters
    ----------
        x, y, z (int / numpy.uint32)
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The parity of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over each passed parameter
    for args in (x, y, z):
        # Use isInstance to ensure correct type (object, desired type)
        # See: https://www.w3schools.com/python/ref_func_isinstance.asp
        if not isinstance(args, (int, np.integer)):
            # Raises / Throws a TypeError with a message
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast all inputs as NumPy unsigned 32-bit integers.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Apply bitwise XOR (^) to the three inputs and return
    return x ^ y ^ z

#### Test Cases (Parity)

Checking
- Correct results for trivial values
- Correct results for non-trivial values
- Type enforcement (raise TypeError for non-integer values)

In [3]:
def test_valid_parity():
    assert parity(1, 0, 0) == np.uint32(1)      # Odd number of 1s
    assert parity(1, 1, 0) == np.uint32(0)      # Even number of 1s
    assert parity(5, 12, 6) == np.uint32(15)    # General (non-trivial) case

def test_parity_type_error():
    try:
        parity(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
test_valid_parity()
test_parity_type_error()

print("All test cases pass")

All test cases pass


### Implementing Choose (Ch)

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2 p.10) as:
>
>$\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x \land z)$
>


**How it works**:  
The ch functions uses x as a selector mask. For each bit position , if x is 1, the output takes the bit from y.  
If x is 0, the output takes the bit from z.  

This is achieved by calculating $(x \land y)$ to select bits from y where x is 1, then $(\lnot x \land z)$ to select bits from z where x is 0. Then XOR the results together.

**Example**  
X       =   1110  
Y       =   1010  
Z       =   1111  

Output  =   1011

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [4]:
def ch(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Choose: return bits from y where x is 1, and bits from z where x is 0.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'masked' value of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over the parameters passed
    for args in (x, y, z):
        # Use isinstance + raise TypeError as before
        # See: Problem 1 -> Implementing Parity
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation and return the value
    return (x & y) ^ (~x & z)

#### Test Cases (Choose)

Checking
- Correct results for trivial values
- Correct results for non-trivial values
- Type enforcement (raise TypeError for non-integer values)

In [5]:
# Same test cases and checks as Problem 1 -> Parity
def test_valid_ch():
    assert ch(1, 0, 0) == np.uint32(0)
    assert ch(1, 1, 0) == np.uint32(1)
    assert ch(5, 12, 6) == np.uint32(6)

def test_ch_type_error():
    try:
        ch(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_valid_ch()
test_ch_type_error()

print("All test cases pass")

All test cases pass


### Implementing Majority (Maj)

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\text{maj}(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)$
>

**How it works**:  
The Maj functions implements majority voting at each bit position. For each bit, the output is 1 if atleast two input bits are 1, otherwise 0.  
This is computed by taking the AND of each pair of inputs $(x \land y)$, $(x \land z)$, and $(y \land z)$, then XOR the three results together.


**Example**  
X       =   11101110  
Y       =   10100101  
Z       =   11111001  

Output  =   11101101

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [6]:
def maj(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Majority: output bit is 1 if two or more input bits are 1.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'voted' values of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over args, use isinstance & raise TypeError as before
    # See: Problem 1 -> Implementing Parity
    for args in (x, y, z):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation & return the value
    return (x & y) ^ (x & z) ^ (y & z)

#### Test Cases (Majority)

Checking
- Correct results for trivial values
- Correct results for non-trivial values
- Type enforcement (raise TypeError for non-integer values)

In [7]:
# Same test cases and checks as Problem 1 -> Implementing Parity
def test_maj_valid():
    assert maj(1, 0, 0) == np.uint32(0)
    assert maj(1, 1, 0) == np.uint32(1)
    assert maj(5, 12, 6) == np.uint32(4)

def test_maj_type_error():
    try:
        maj(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_maj_valid()
test_maj_type_error()

print("All test cases pass")


All test cases pass


### Implementing Big & Small Sigmas with Helper Functions

#### Helper Functions

The Sigma functions use bit rotation and shifting operations. These helper functions implement ROTR & SHR as defined in FIPS 180-4, Section 3.2 (p.9).

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, p.8-9) as:
>
>$\text{ROTR}^n(x) = (x \gg n) \lor (x \ll (w-n))$
>
>$\text{SHR}^n(x) = x \gg n$
>

- `rotr(x, n)`: Rotate right -  shifts bits to the right **n** times, 'fallen' bits move back to the left (start)
- `shr(x, n)`: Logical shift right - shifts bits to the right **n** times; fills left with zeros.

**Example**  
x = 10110011

rotr(x, 2) = 11101100  
shr(x, 4) = 00001011

See: [FIPS 180-4, Section 3.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [8]:
def rotr(x: int, n: int) -> np.uint32:
    """
    Perform a right rotation on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int
            32-bit unsigned integer to be rotated.
        n : int
            Number of positions to rotate.

    Returns
    -------
        numpy.uint32
            The result of the right rotation as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the rotation and return the value
    return (x >> n) | (x << (32 - n))

In [9]:
def shr(x: int, n: int) -> np.uint32:
    """
    Perform a logical right shift on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int 
            32-bit unsigned integer to be shifted.
        n : int
            Number of positions to shift.

    Returns
    -------
        numpy.uint32
            The result of the logical right shift as a 32-bit unsigned integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
        
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the logical right shift and return the value
    return x >> n

##### Test Cases (Helper Functions)

Checking
- Correct results for passed values
- Type enforcement (raise TypeError for non-integer values)

In [10]:
# Same format for tests as previous sub-problems
def test_rotr_valid():
    assert rotr(10, 1) == np.uint32(5)
    assert rotr(10, 2) == np.uint32(2147483650)
    assert rotr(16, 4) == np.uint32(1)

def test_shr_valid():
    assert shr(10, 1) == np.uint32(5)
    assert shr(10, 2) == np.uint32(2)

def test_rotr_type_error():
    try:
        rotr(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

def test_shr_type_error():
    try:
        shr(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_rotr_valid()
test_shr_valid()
test_rotr_type_error()
test_shr_type_error()

print("All test cases pass")

All test cases pass


#### Big Sigma 0 - $\Sigma_0(x)$

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\Sigma_0(x) = \text{ROTR}^{2}(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)$
>  

**How it works**:  
Rotates the input x by three different amounts (2, 13 and 22 bit positions), then XOR the three results together. This makes sure the bit mixing is rigorous.  

The rotation amounts were chosen during SHA-256's development process to ensure optimal security.

**Example**  
x = 10110011

Each bit is rotated **2**, **13** & **22** times, then XOR'd together | (2 XOR 13) XOR 22.  

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)


In [11]:
def big_sigma_0(x: int) -> np.uint32:
    """
    Implementing the Big Sigma 0 function used in SHA-256.

    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of ROTR(2) XOR ROTR(13) XOR ROTR(22)

    Raises
    ------
        TypeError
            If the argument is not an integer.
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type
    x = np.uint32(x)

    # Perform the Big Sigma 0 operation and return the value
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))

#### Big Sigma 1 - $\Sigma_1(x)$

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\Sigma_1(x) = \text{ROTR}^{6}(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)$
>  

**How it works**:  
Similar to $\Sigma_0$. Rotates the input x by three different amounts (6, 11 and 25 bit positions), then XOR the three results together. This makes sure the bit mixing is rigorous.  

The rotation amounts were chosen during SHA-256's development process to ensure optimal security.
**Example**  
x = 10110011  

Each bit is rotated **6**, **11** & **25** times, then XOR'd together | (6 XOR 11) XOR 25. 


See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [12]:
def big_sigma_1(x: int) -> np.uint32:
    """
    Implementing the Big Sigma 1 function used in SHA-256.
        
    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of the Big Sigma 1 function as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If the argument is not an integer.
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Big Sigma 1 operation and return the value
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

##### Test Cases (Big Sigma(s))

Checking:
- Correct mathematical implementation by comparing with manual calculations using helper functions
- Type enforcement (raise TypeError for non-integer values)
- Validation for both Big Sigma 0 and Big Sigma 1 functions

In [13]:
def test_bsigma_0_valid():
    assert big_sigma_0(1)  == np.uint32(rotr(1, 2) ^ rotr(1, 13) ^ rotr(1, 22))
    assert big_sigma_0(5)  == np.uint32(rotr(5, 2) ^ rotr(5, 13) ^ rotr(5, 22))
    assert big_sigma_0(10) == np.uint32(rotr(10, 2) ^ rotr(10, 13) ^ rotr(10, 22))

def test_bsigma_1_valid():
    assert big_sigma_1(1)  == np.uint32(rotr(1, 6) ^ rotr(1, 11) ^ rotr(1, 25))
    assert big_sigma_1(5)  == np.uint32(rotr(5, 6) ^ rotr(5, 11) ^ rotr(5, 25))
    assert big_sigma_1(10) == np.uint32(rotr(10, 6) ^ rotr(10, 11) ^ rotr(10, 25))


def test_bsigma_type_error():
    try:
        big_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        big_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_bsigma_0_valid()
test_bsigma_1_valid()
test_bsigma_type_error()

print("All test cases pass")

All test cases pass


### Small Sigma 0 - $\sigma_0(x)$  

**Specification** (FIPS 180-4, Section 4.1.2, p.10):
>
> $\sigma_0(x) = \text{ROTR}^{7}(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^{3}(x)$
>  

**How it works**:  
Combines two rotations (7 and 18 bit positions) with one logical right shift (3 positions), then XORs all three results together. Unlike the Big Sigma functions which only use rotation, this function includes a shift operation that discards bits, creating a different mixing pattern.

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [14]:
def small_sigma_0(x: int) -> np.uint32:
    """
    Implementing the Small Sigma 0 function used in SHA-256.

    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of the Small Sigma 0 function as a 32-bit unsigned integer.

    Raises
    ------
        TypeError
            If the argument is not an integer.
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")
    
    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Small Sigma 0 operation and return the value
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))

### Small Sigma 1 - $\sigma_1(x)$

**Specification** (FIPS 180-4, Section 4.1.2, p.10):
>
> $\sigma_1(x) = \text{ROTR}^{17}(x) \oplus \text{ROTR}^{19}(x) \oplus \text{SHR}^{10}(x)$
>  

**How it works**:  
Combines two rotations (17 and 19 bit positions) with one logical right shift (10 positions), then XORs all three results together. Like $\sigma_0$, this uses both rotation and shift operations for bit mixing.

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [15]:
def small_sigma_1(x: int) -> np.uint32:
    """
    Implementing the Small Sigma 1 function used in SHA-256.

    Parameters
    ----------
        x : int 
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of the Small Sigma 1 function as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If the argument is not an integer.
    """

    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Small Sigma 1 operation and return the value
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

##### Test Cases (Small Sigma(s))

Checking:
- Correct mathematical implementation by comparing with manual calculations using helper functions
- Type enforcement (raise TypeError for non-integer values)
- Validation for both Small Sigma 0 and Small Sigma 1 functions

In [16]:
def test_ssigma_0_valid():
    assert small_sigma_0(1)  == np.uint32(rotr(1, 7) ^ rotr(1, 18) ^ shr(1, 3))
    assert small_sigma_0(5)  == np.uint32(rotr(5, 7) ^ rotr(5, 18) ^ shr(5, 3))
    assert small_sigma_0(10) == np.uint32(rotr(10, 7) ^ rotr(10, 18) ^ shr(10, 3))

def test_ssigma_type_error():
    try:
        small_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        small_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_ssigma_0_valid()
test_ssigma_type_error()

print("All test cases pass")

All test cases pass


## Problem 2 - Fractional Parts of Cube Roots

### Implementing Primes

**What is a prime number** A prime number is divisible by itself and 1 only.

**Spec.** SHA-256 derives constants based off prime numbers.

Constants listed in the Secure Hash Standard  
See: [FIPS 180-4, Sections 4.2.2, p.11](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

  
**How it works**:  
- Performs input & type validation
- Create a list with first prime (2)
- Loops until the list of `primes` is length of `n`
- Checks if the number is divisible by any of the current primes found
- Jumps 2 numbers to check at a time (all even numbers > 2 aren't prime)



References:  
See: [GeeksForGeeks Code Example](https://www.geeksforgeeks.org/dsa/generate-and-print-first-n-prime-numbers/)


In [17]:
def primes(n: int) -> list[int]:
    """
    Generate the first n prime numbers.

    Parameters
    ----------
        n : int

    Returns
    -------
        list[int]
            A list of the first n prime numbers.

    Raises
    ------
        TypeError
            If the argument is not an integer.
        ValueError
            If the argument is not a positive integer.
    """
    # Type check as before
    if not isinstance(n, (int, np.integer)):
        raise TypeError("Argument must be an integer.")
    
    # Ensure n is a positive integer
    if n <= 0:
        raise ValueError("Argument must be a positive integer.")

    # Start with the first prime number in a list
    primes = [2]

    # Start checking for primes from 3 onwards
    num = 3

    # Loop until we have calculated n primes
    while len(primes) < n:
        # Assume number is prime to start
        is_prime = True

        for i in range(len(primes)):
            # Check to see if current number is divisible by any previous primes
            if num % primes[i] == 0:
                # If it is, its not a prime number, skip to next number
                is_prime = False
                break
        
        # If number is still prime, add to list
        if is_prime:
            primes.append(num)

        # Skip even numbers, even numbers above 2 are not prime
        num += 2  

    # Return completed list
    return primes

#### Testing Prime

- Tests to see if the function can return the first N Prime numbers.
- Ensure the correct errors are being raised for passed args.

In [18]:
def test_prime_valid():
    assert primes(5) == [2, 3, 5, 7, 11]
    assert primes(10) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

def test_prime_type_value_error():
    try:
        primes(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        primes(-5)
    except ValueError:
        pass
    else:
         raise AssertionError("Expected ValueError for negative input")
    
test_prime_valid()
test_prime_type_value_error()

print("All test cases pass")

All test cases pass


### Calculating Cube Roots

**What is a cube root?** The cube root of a number x is the value that, when multiplied by itself three times, equals x.  

**Why cube roots?** SHA-256 uses the first 32 bits of the fractional parts of cube roots of the first 64 primes as round constants (K values). These mathematically-derived values follow the "nothing up my sleeve" principle, providing assurance against intentional weaknesses.

**The process:**
1. Calculate the cube root of each prime
2. Extract the fractional part (the digits after the decimal: 0.2599...)
3. Convert to 32-bit representation by multiplying by 2^32 and truncating to integer
4. Display as 8-character hexadecimal string

This matches the first K constant in Secure Hash Standard

See: [FIPS 180-4, Section 4.2.2, p.11](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [19]:
def calc_cube_roots(n: int) -> list[str]:
    """
    Calculate the cube roots of a list of prime numbers.

    Parameters
    ----------
        n : int
            The number of cube roots to calculate.

    Returns
    -------
        list[str]
            A list of hexadecimal strings representing the cube roots.

    Raises
    ------
        TypeError
            If the argument is not a list of integers.
    """
    # Type check as before
    if not isinstance(n, (int, np.integer)):
        raise TypeError("Argument must be an integer.")

    # Get the first n primes from function above
    primes_list = primes(n)

    # Create an empty list to store results
    hex_cube_roots = []

    # Loop over each prime number
    for p in primes_list:
        # Calculate the cube root using numpy.cbrt
        # See: https://numpy.org/doc/stable/reference/generated/numpy.cbrt.html
        cube_root = np.cbrt(p)

        # Extract the fractional part of the cube root
        fractional_part = cube_root % 1

        # Convert the fractional number to 32-bit by multiplying by 2^32
        full_32bit = int(fractional_part * (2**32))

        # Convert to hexadecimal and format to 8 characters
        # See: https://docs.python.org/3/library/functions.html#hex
        hex_value = format(full_32bit, '08x')

        # Append to results list
        hex_cube_roots.append(hex_value)

        # Print the results
        print(hex_value)

    # Return the list of strings
    return hex_cube_roots

#### Test Cases (Cube Roots)

**Testing**:
- Correct calculation of cube roots for first 64 primes
- Proper hexadecimal formatting (8 characters each)
- Type enforcement for invalid inputs
- Validation against SHA-256 K constants 

See: [FIPS 180-4, Section 4.2.2, p.11](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf)


In [20]:
def test_cube_roots():
    # SHA-256 round constants (first 64 cube roots of primes)
    # See: FIPS 180-4, Section 4.2.2, p.11
    # https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf
    secure_hash_constants = ["428a2f98", "71374491", "b5c0fbcf", "e9b5dba5", "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5",
        "d807aa98", "12835b01", "243185be", "550c7dc3", "72be5d74", "80deb1fe", "9bdc06a7", "c19bf174",
        "e49b69c1", "efbe4786", "0fc19dc6", "240ca1cc", "2de92c6f", "4a7484aa", "5cb0a9dc", "76f988da",
        "983e5152", "a831c66d", "b00327c8", "bf597fc7", "c6e00bf3", "d5a79147", "06ca6351", "14292967",
        "27b70a85", "2e1b2138", "4d2c6dfc", "53380d13", "650a7354", "766a0abb", "81c2c92e", "92722c85",
        "a2bfe8a1", "a81a664b", "c24b8b70", "c76c51a3", "d192e819", "d6990624", "f40e3585", "106aa070",
        "19a4c116", "1e376c08", "2748774c", "34b0bcb5", "391c0cb3", "4ed8aa4a", "5b9cca4f", "682e6ff3",
        "748f82ee", "78a5636f", "84c87814", "8cc70208", "90befffa", "a4506ceb", "bef9a3f7", "c67178f2"
    ]

    calculated_constants = calc_cube_roots(64)
    assert calculated_constants == secure_hash_constants, "Calculated constants do not match expected values."

def test_cube_roots_type_error():
    try:
        calc_cube_roots(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_cube_roots()
test_cube_roots_type_error()
print("All test cases pass")

428a2f98
71374491
b5c0fbcf
e9b5dba5
3956c25b
59f111f1
923f82a4
ab1c5ed5
d807aa98
12835b01
243185be
550c7dc3
72be5d74
80deb1fe
9bdc06a7
c19bf174
e49b69c1
efbe4786
0fc19dc6
240ca1cc
2de92c6f
4a7484aa
5cb0a9dc
76f988da
983e5152
a831c66d
b00327c8
bf597fc7
c6e00bf3
d5a79147
06ca6351
14292967
27b70a85
2e1b2138
4d2c6dfc
53380d13
650a7354
766a0abb
81c2c92e
92722c85
a2bfe8a1
a81a664b
c24b8b70
c76c51a3
d192e819
d6990624
f40e3585
106aa070
19a4c116
1e376c08
2748774c
34b0bcb5
391c0cb3
4ed8aa4a
5b9cca4f
682e6ff3
748f82ee
78a5636f
84c87814
8cc70208
90befffa
a4506ceb
bef9a3f7
c67178f2
All test cases pass


## Problem 3 - Padding

### Implementing Block Parse

**Purpose:** SHA-256 processes messages in fixed-size blocks of 512 bits (64 bytes). Messages of arbitrary length must be padded to ensure the final length is a multiple of 512 bits. This function implements the padding scheme and yields message blocks one at a time.

**What is padding?** Padding adds extra data to a message to reach a required length. For SHA-256, padding ensures:
1. The message ends on a block boundary (multiple of 512 bits)
2. The original message length is preserved and authenticated
3. Different messages produce different padded results (no collisions from padding alone)

**Specification** (FIPS 180-4, Sections 5.1.1 and 5.2.1):

The padding process:
1. Append a single '1' bit (byte `0x80` = `10000000` in binary)
2. Append '0' bits until message length ≡ 448 mod 512 (leaves 64 bits remaining)
3. Append original message length as 64-bit big-endian integer
4. Split padded message into 512-bit blocks

**Why this padding scheme?** 
- The '1' bit ensures there's always at least some padding (prevents ambiguity)
- The length field prevents length-extension attacks
- The specific modulo ensures exactly 64 bits remain for the length

**Example:**  
  
Original message: "abc" (24 bits)  
Step 1 - Append '1' bit:     "abc" + 0x80  
Step 2 - Append zeros:        Pad to 448 bits (56 bytes)  
Step 3 - Append length:       Add 0x0000000000000018 (24 in 64-bit)  
Result: Single 512-bit block  
  
**Generator function:** This implementation uses a Python generator, which yields blocks one at a time rather than returning all blocks at once. This is memory-efficient for large messages.

**Jargon explanation:**
- **Generator**: A function that uses `yield` to produce values one at a time, pausing between each
- **Big-endian**: Most significant byte first (standard network byte order)
- **mod 512**: Remainder when divided by 512

See: [FIPS 180-4, Sections 5.1.1 and 5.2.1](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [21]:
def block_parse(msg: bytes) -> Generator[bytes, None, None]:
    """
    Parse a message into 512-bit blocks with proper SHA-256 padding.

    Parameters
    ----------
        msg : bytes
            The input message as a byte string.

    Yields
    ------
        bytes
            The padded message as a byte string, length is a multiple of 64 bytes (512 bits).
    
    Raises
    ------
        TypeError
            If the argument is not a byte string.
    """

    # Type check as before
    if not isinstance(msg, bytes):
        raise TypeError("Argument must be a byte string.")

    # Get the original message length in bits
    origin_bit_len = len(msg) * 8

    # Append the bit '1' to the message (0x80 = 10000000 in binary)
    # See: FIPS 180-4, Section 5.1.1, p.15
    # https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf
    msg += b'\x80'

    # Append '0' bits until message length ≡ 448 mod 512
    # This leaves exactly 64 bits (8 bytes) for the length field
    while (len(msg) % 64) != 56:  # 56 bytes = 448 bits
        msg += b'\x00'
        
    # Append the original message length as a 64-bit big-endian integer
    # See: FIPS 180-4, Section 5.1.1
    # https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf
    msg += origin_bit_len.to_bytes(8, byteorder='big')

    # Now msg length is a multiple of 512 bits (64 bytes)
    # Yield each 512-bit block
    for i in range(0, len(msg), 64):  # 64 bytes = 512 bits
        yield msg[i:i+64]


### Test Cases (Padding)  
- Correct padding for messages of various lengths (short messages, messages near block boundaries)
- Proper block size validation (all output blocks are exactly 64 bytes/512 bits)
- Type enforcement for non-bytes input and verification against known test vectors

In [22]:
def test_block_parse():
    # Test case 1: Empty message
    blocks = list(block_parse(b""))
    assert len(blocks) == 1
    assert len(blocks[0]) == 64  # One block of 512 bits
    assert blocks[0][-8:] == (0).to_bytes(8, byteorder='big')  # Length field is 0

    # Test case 2: Short message
    msg = b"abc"
    blocks = list(block_parse(msg))
    assert len(blocks) == 1
    assert len(blocks[0]) == 64
    expected_length = len(msg) * 8
    assert blocks[0][-8:] == expected_length.to_bytes(8, byteorder='big')

    # Test case 3: Message that requires multiple blocks
    msg = b"a" * 100  # 800 bits
    blocks = list(block_parse(msg))
    assert len(blocks) == 2
    assert all(len(block) == 64 for block in blocks)
    expected_length = len(msg) * 8
    assert blocks[-1][-8:] == expected_length.to_bytes(8, byteorder='big')

    # Test case 4: Type enforcement
    try:
        list(block_parse("not bytes"))  # Should raise TypeError
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for non-bytes input")

test_block_parse()
print("All test cases pass")

All test cases pass


## Problem 4 - Hashes

### Implementing Initial Hash Values - Helper Function

**Purpose:** SHA-256 requires eight initial hash values (H⁽⁰⁾) to begin the hashing process. These serve as the starting state before processing any message blocks. The `hash(current, block)` function needs an intial starting point when processing the first block of a message.

**What are initial hash values?** These are the starting constants used in the SHA-256 algorithm. Like the K constants from Problem 2, they are derived to provide assurance against intentional backdoors.

**Specification** (FIPS 180-4, Section 5.3.3, p.15):

The eight initial hash values are the first 32 bits of the fractional parts of the square roots of the first 8 prime numbers:
```
H⁽⁰⁾₀ = 0x6a09e667    (√2)
H⁽⁰⁾₁ = 0xbb67ae85    (√3)
H⁽⁰⁾₂ = 0x3c6ef372    (√5)
H⁽⁰⁾₃ = 0xa54ff53a    (√7)
H⁽⁰⁾₄ = 0x510e527f    (√11)
H⁽⁰⁾₅ = 0x9b05688c    (√13)
H⁽⁰⁾₆ = 0x1f83d9ab    (√17)
H⁽⁰⁾₇ = 0x5be0cd19    (√19)
```

**Why square roots?** Square roots are used for initial values while cube roots are used for round constants (K values from Problem 2). Both follow the same mathematical derivation principle to ensure the constants are verifiably random.

See: [FIPS 180-4, Section 5.3.3](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [ ]:
def inital_hash_values() -> list[np.uint32]:
    """
    Return the initial hash values for SHA-256 as a list of 32-bit unsigned integers.

    Returns
    -------
        list[numpy.uint32]
            A list of the initial hash values for SHA-256.
    """

    # SHA-256 initial hash values
    # See: FIPS 180-4, Section 5.3.3, p.16
    # https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf
    H = [
        np.uint32(0x6a09e667),  # √2
        np.uint32(0xbb67ae85),  # √3
        np.uint32(0x3c6ef372),  # √5
        np.uint32(0xa54ff53a),  # √7
        np.uint32(0x510e527f),  # √11
        np.uint32(0x9b05688c),  # √13
        np.uint32(0x1f83d9ab),  # √17
        np.uint32(0x5be0cd19)   # √19
    ]
    
    return H

## Problem 5 - Passwords

## Conclusion

## End